In [ ]:
import cvrplib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from deap import base, algorithms
from deap import creator
from deap import tools
 
import random

In [ ]:
# Read instances
instance = cvrplib.read('../Downloads/A/A-n61-k9.vrp')

# Download instances
instance,solution = cvrplib.download('A-n33-k6', solution = True)

# List instance names 
cvrplib.list_names()                      # All instance names
cvrplib.list_names(low=0, high=200)     # Instances with between [100, 200] customers
cvrplib.list_names(vrp_type='vrptw')      # Only VRPTW instances

In [ ]:
new = []
for i in solution.routes:
    new += i + [0]

new

In [ ]:
plt.figure(dpi=100)
plt.scatter(*instance.coordinates[instance.depot],color = "red")
for i in instance.customers:
    plt.scatter(*instance.coordinates[i],color = "green")
    
plt.show()

In [ ]:
Trucks = int(instance.name.split('-')[2][1::])
Capacity = Trucks*[instance.capacity]
depo_ind = instance.depot
Demands = instance.demands
Customers = instance.customers
Ways = instance.distances
locWay = [0]
Path = []
Uvisited = [i for i in range(1,len(Customers) + 1)]

In [ ]:
Uvisited

In [ ]:
def Initial_solution(instance):
    Trucks = int(instance.name.split('-')[2][1::])
    Capacity = Trucks*[instance.capacity]
    depo_ind = instance.depot
    Demands = instance.demands
    Customers = instance.customers
    Ways = instance.distances
    locWay = [0]
    Path = []
    Uvisited = [i for i in range(1,len(Customers) + 1)]
    curr = 0
    nxt = -1;
    k = 1000000;
    Cost = 0;
    sol = [];
    for track in range(Trucks):
        locWay = [];
        while len(Uvisited) > 0:
            for i in range(len(Ways[curr])):
                if Demands[i] < k and Ways[curr][i] != 0 and (i) in Uvisited:
                    k = Demands[i]
                    nxt = i
            k = 1000000;
            if  Capacity[track] - Demands[nxt] >= 0:  
                Cost += Ways[curr][nxt]
                curr = nxt
                Capacity[track] = Capacity[track] - Demands[nxt]
                locWay = locWay + [nxt];
                Uvisited.remove(nxt); 
                k = 100000;
            else:
                Cost += Ways[curr][0]
                Path.append(locWay + [0])
                sol += (locWay + [0])
                curr = 0
                k = 100000;
                break;
            
            if len(Uvisited) == 0:
                Cost += Ways[curr][0]
                Path.append(locWay + [0])
                sol += (locWay + [0])
                break;
    return sol,Path,Cost


In [ ]:
sol, K, Cost = Initial_solution(instance)

In [ ]:
K,u

In [ ]:
def plotik(K):
    o = 0
    x = [instance.coordinates[instance.depot][0]]
    y = [instance.coordinates[instance.depot][1]]
    plt.figure(dpi=100)
    colors =["green","blue","grey","orange","red","pink","cyan","olive","brown","black"];
    Tracks = ["Route 1","Route 2","Route 3","Route 4","Route 5","Route 6","Route 7","Route 8"];
    markers = []
    for i in K:
        x += [instance.coordinates[i][0]]
        y += [instance.coordinates[i][1]]
        #print(x,y)
        #markers += Tracks[o]
        #plt.legend(markers)
        plt.plot(x,y, color=colors[o])
        plt.scatter(x,y, color="blue")
        if i == instance.depot:
            o += 1
            print(x,len(x))
            x = [instance.coordinates[i][0]]
            y = [instance.coordinates[i][1]]
    plt.scatter(*instance.coordinates[instance.depot],color = "black")
    plt.show() 

In [ ]:
plotik(sol)
new = []
for i in solution.routes:
    new += i + [0]
    
plotik(new),F(sol),F(new), Uvisited

In [ ]:
T0 = 10000000
from random import shuffle

def F(x):
    Cost = 0;
    curr = 0;
    for i in x:
        Cost += Ways[curr][i]
        curr = i
    return Cost

def new_Way(K):
    tranform = []
    summ = 0
    
    for i in K:
        if i != 0:
            tranform += [i]
        
    
    New_Way = []
    [i,j] = sorted(random.sample(range(len(tranform)),2))
    updateTour = tranform[:i] + tranform[j:j+1] + tranform[i+1:j] + tranform[i:i+1] + tranform[j+1:]
    shuffle(tranform)
    #updateTour = tranform
    #print(updateTour)
    N = []
    curr = 0
    for i in updateTour:
        if summ + Demands[i] <= instance.capacity:
            N += [i]
            summ += Demands[i]
            curr = i
        else:
            summ = Demands[i]
            N += [0]
            N += [i]
            curr = i
    
    N += [0]
        
    return N

def check(sol, cap):
    summ = 0
    
    if sol[len(sol) - 1] == 0 and sol[0] == 0:
        return False
    
    for i in range(len(sol) - 1):
        if sol[i] == sol[i + 1]:
            return False
    
    curr = 0
    for i in range(len(sol)):
        summ += Ways[curr][sol[i]]
        if sol[i] == instance.depot:
            if summ > cap:
                return False
            
            else:
                summ = 0
        
        curr = i
    
    return True

def algorithm(instance):
    sol ,_,_ = Initial_solution(instance)
    T = 100
    alpha = 0.9
    curr = 0
    Fbest = F(sol)
    Xbest = sol
    iter_curr = 0
    iter_max = 1000
    d = 0
    while T > 0.1:
        iter_curr = 0
        while iter_curr <= iter_max:
            new_W = new_Way(sol) 
            if F(new_W) - F(Xbest) <= 0:
                sol = new_W
                if F(sol) < Fbest:
                    Xbest = sol
                    Fbest = F(sol)
                else:
                    new_W = new_Way(sol)
                
            else:
                if np.exp(F(sol) - F(new_W)) / T > random.random():
                    sol = new_W
                
            iter_curr += 1
        T *= alpha
    
    return Xbest, Fbest
            

def p(x,y, temp):
    if np.exp(F(y) - F(x)) / temp > random.random():
        return True

In [ ]:
# Read instances
instance = cvrplib.read('../Downloads/E/E-n76-k10.vrp')

# Download instances
instance,solution = cvrplib.download('E-n76-k10', solution = True)

# List instance names 
cvrplib.list_names()                      # All instance names
cvrplib.list_names(low=0, high=200)     # Instances with between [100, 200] customers
cvrplib.list_names(vrp_type='vrptw')      # Only VRPTW instances



start_time = time.time()
h,l = algorithm()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from os import walk

filenames = next(walk("../Downloads/B"), (None, None, []))[2]  

In [ ]:
filenames[0].split('.')[1]

In [ ]:
for k in [next(walk("../Downloads/A"), (None, None, []))[2]  ,next(walk("../Downloads/B"), (None, None, []))[2] ,next(walk("../Downloads/E"), (None, None, []))[2] ]:
    for names in k:
        if names.split('.')[1] != 'sol':
            instance,solution_1 = cvrplib.download(names[:len(names)-4], solution = True)
            Trucks = int(instance.name.split('-')[2][1::])
            Capacity = Trucks*[instance.capacity]
            depo_ind = instance.depot
            Demands = instance.demands
            Customers = instance.customers
            Ways = instance.distances
            locWay = [0]
            Path = []
            Uvisited = [i for i in range(1,len(Customers) + 1)]
            start_time = time.time()
            h,l = algorithm(instance)
            new = []
            for i in solution_1.routes:
                new += i + [0]
            print(names,F(h),F(new),(time.time() - start_time),(F(new)/F(h))*100)
            #if F(h) < F(new):
            #    print("*****",names)
            #    plotik(h)
            #    plotik(new)
            #    print("*****")
            

In [ ]:
for k in [next(walk("../Downloads/A"), (None, None, []))[2] ]:
    for names in k:
        if names.split('.')[1] != 'sol':
            instance,solution_1 = cvrplib.download(names[:len(names)-4], solution = True)
            Trucks = int(instance.name.split('-')[2][1::])
            Capacity = Trucks*[instance.capacity]
            depo_ind = instance.depot
            Demands = instance.demands
            Customers = instance.customers
            Ways = instance.distances
            locWay = [0]
            Path = []
            Uvisited = [i for i in range(1,len(Customers) + 1)]
            start_time = time.time()
            h,l = algorithm(instance)
            new = []
            for i in solution_1.routes:
                new += i + [0]
            
            print(names,F(h),F(new),(time.time() - start_time),(F(new)/F(h))*100)
            print("*****",names)
            plotik(h)
            plotik(new)
            print("*****")
            